네거티브 샘플링을 이용한 Word2Vec 구현

네거티브 샘플링이란? (Negative Sampling)
Word2Vec의 출력층에서 소프트맥스 함수를 지난 벡터와 실제값인 원-핫 벡터와의 오차를 구하고 임베딩 테이블에 있는 모든 단어에 대한 임베딩 벡터 값을 업데이트한다. 하지만 단어 집합의 크기가 커질 수록 무거운 작업이된다.

강아지 고양이나 귀여운을 가지고 학습해야하는데, 돈가스나 컴퓨터와 같은 단어들 까지 포함해서 업데이트 하는 것은 비효율적이다.
따라서 전체 단어 집합보다 훨씬 더 작은 단어 집합을 만들어놓고 마지막 단계를 이진 분류문제로 변환한다. 주변 단어들을 긍정, 랜덤으로 샘플링된 단어들을 부정으로 레이블링하면 이진 분류 문제를 위한 데이터셋이 된다. 즉 학습하는 단어 집합을 줄일 수 있다.

다양한 단어를 집어넣고 같이 나오면 positive, 같이 안나오면 negative로 나오게 한다.
20 뉴스 그룹 데이터로 실습을 해보자 !


In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.datasets import fetch_20newsgroups
from tensorflow.keras.preprocessing.text import Tokenizer

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers','footers','quotes'))
documents = dataset.data
print('총 샘플 갯수',len(documents))

총 샘플 갯수 11314


In [2]:
#불필요한 토큰을 제거하고 소문자화를 통해서 정규화를 진행
news_df = pd.DataFrame({'document':documents})
#특수문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]"," ")
#길이가 3이하인 단어는 제거
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
#전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())
#빈값(empty)를 NaN으로 변환
news_df.replace("",float("NaN"),inplace=True)
news_df.isnull().values.any()

True

In [3]:
news_df.dropna(inplace=True)
print('전처리 후 총 샘플 수',len(news_df))

전처리 후 총 샘플 수 11004


In [4]:
#불용어 제거
stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
tokenized_doc = tokenized_doc.to_list()

#모든 샘플 중 단어가 1개 이하인 경우 제거
# 단어가 1개 이하인 샘플의 인덱스를 찾아서 저장하고, 해당 샘플들은 제거.
drop_train = [index for index, sentence in enumerate(tokenized_doc) if len(sentence) <= 1]
tokenized_doc = np.delete(tokenized_doc, drop_train, axis=0)
print('총 샘플 수 :',len(tokenized_doc))

총 샘플 수 : 10961


c:\Users\HOME\anaconda3\lib\site-packages\numpy\lib\function_base.py:5071: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)


In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokenized_doc)

word2idx = tokenizer.word_index

idx2word = {value: key for key, value in word2idx.items()}
encoded = tokenizer.texts_to_sequences(tokenized_doc)

print(encoded[:2])

[[40, 53, 927, 143, 15889, 1684, 546, 279, 871, 12028, 17773, 24007, 29726, 279, 871, 63435, 871, 1128, 1103, 1998, 851, 29727, 913, 731, 20477, 279, 871, 170, 143, 1811, 149, 279, 20478, 17773, 6645, 5710, 76, 63436, 7, 36, 165, 614, 653, 29728, 6911, 24008, 2082, 829, 17774, 1119, 8790, 355, 1072, 15890, 671, 57, 163, 4231, 7206, 1933, 440, 56, 282, 4730, 9275, 2690, 39306], [1283, 429, 3, 52, 6164, 159, 112, 474, 89, 17775, 18, 63, 4731, 2865, 63437, 1042, 402, 39307, 8791, 902, 44, 8328, 316, 13041, 902, 3452, 5923, 533, 18, 87, 4732, 9872, 160, 1403, 120, 151, 5194, 63438, 63439, 17776, 63440, 13041, 903, 63441, 63442, 11172, 17777]]


전처리 한 데이터셋을 통해서 네거티브 샘플링을 한다.
네거티브샘플링을 위해서 케라스에서 제공하는 전처리 도구인 skipgrams를 사용한다.
시간상 상위 10개의 뉴스그룹 샘플에 대해서만 수행해보도록 하자!

In [6]:
from tensorflow.keras.preprocessing.sequence import skipgrams
#네거티브 샘플링
vocab_size = len(word2idx) + 1 

skip_grams = [skipgrams(sample, vocabulary_size=vocab_size,window_size=10) for sample in encoded[:10]]

In [7]:
pairs, labels = skip_grams[0][0], skip_grams[0][1]
for i in range(5):
    print("({:s} ({:d}), {:s} ({:d})) -> {:d}".format(
          idx2word[pairs[i][0]], pairs[i][0], 
          idx2word[pairs[i][1]], pairs[i][1], 
          labels[i]))

(whole (170), texts- (111577)) -> 0
(realize (851), oneida (178139)) -> 0
(jews (282), government (57)) -> 1
(israels (17773), geobench* (64239)) -> 0
(reputation. (24007), realize (851)) -> 1


위와 같이 어떤게 같이 등장하는지 같이 등장하지 않는지 알 수 있다.
이 작업을 모든 뉴스그룹 샘플에 대해서 수행해보자 

In [8]:
skip_grams = [skipgrams(sample, vocabulary_size=vocab_size, window_size=10) for sample in encoded]

In [9]:
#Skip-Gram with negative sampling(SGNS) 구현하기
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Reshape, Activation, Input, Dot
from tensorflow.keras.utils import plot_model
from IPython.display import SVG

#임베딩 벡터의 차원은 100으로 두고 두개의 임베딩 층을 추가한다.
embedding_dim = 100

#중심 단어를 위한 임베딩 테이블
w_input = Input(shape=(1,),dtype='int32')
word_embedding = Embedding(vocab_size,embedding_dim)(w_input)

#주변 단어를 위한 임베딩 테이블
c_input = Input(shape=(1,),dtype='int32')
context_embedding = Embedding(vocab_size,embedding_dim)(c_input)


각 임베딩 테이블은 중심단어와 주변단어 각각을 위한 임베딩 테이블이고 각 단어는 임베딩 테이블을 거쳐서 내적을 수행하고 내적의 결과는 1또는 0을 예측해야하므로 시그모이드 함수를 활성화 함수로 거쳐서 최종 예측값을 얻는다.

In [14]:
dot_product = Dot(axes=2)([word_embedding,context_embedding])
dot_product = Reshape((1,),input_shape=(1,1))(dot_product)
output = Activation('sigmoid')(dot_product)

model = Model(inputs=[w_input,c_input],outputs=output)
model.summary()
model.compile(loss='binary_crossentropy',optimizer='adam')
plot_model(model,to_file='model3.png',show_shapes=True,show_layer_names=True,rankdir='TB')


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1, 100)       18183900    ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 1, 100)       18183900    ['input_2[0][0]']                
                                                                                            

In [15]:
for epoch in range(1,6):
    loss = 0
    for _,elem in enumerate(skip_grams):
        first_elem = np.array(list(zip(*elem[0]))[0],dtype='int32')
        second_elem = np.array(list(zip(*elem[0]))[1],dtype='int32')
        labels = np.array(elem[1],dtype='int32')
        X = [first_elem,second_elem]
        Y = labels
        loss += model.train_on_batch(X,Y)
    print('Epoch : ',epoch,'loss : ',loss)

Epoch :  1 loss :  5589.219235010445
Epoch :  2 loss :  4294.254311650991
Epoch :  3 loss :  4028.353681370616
Epoch :  4 loss :  3723.0173838641495


KeyboardInterrupt: 